# 스파크를 이용해 대용량 모빌리티 데이터 읽기

In [9]:
import os
import sys
import pandas as pd

## 1. 판다스로 읽기

In [10]:
filename = os.getcwd() + '/data/fhvhv_tripdata_2020-07.parquet'

In [11]:
%%time
data = pd.read_parquet(filename)

CPU times: user 3.4 s, sys: 1.41 s, total: 4.81 s
Wall time: 2.08 s


In [12]:
# 'hvfhs_license_num' 면허 번호 
# 'dispatching_base_num' 지역 라이센스 번호
# 'pickup_datetime' 승차 시간
# 'dropoff_datetime' 하차시간
# 'PULocationID' 승차지역ID
# 'DOLocationID' 하차지역ID 'trip_miles'
# 'shared_request_flag' 합승여부 Flag
       
data.head(3)

,hvfhs_license_num,dispatching_base_num,originating_base_num,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,...,sales_tax,congestion_surcharge,airport_fee,tips,driver_pay,shared_request_flag,shared_match_flag,access_a_ride_flag,wav_request_flag,wav_match_flag
0,HV0003,B02764,B02764,2020-06-30 23:56:13,2020-06-30 23:58:08,2020-07-01 00:00:21,2020-07-01 00:12:27,143,107,3.37,...,1.27,2.75,NaN,0.0,12.28,N,N,,N,N
1,HV0003,B02764,B02764,2020-07-01 00:18:14,2020-07-01 00:20:03,2020-07-01 00:22:39,2020-07-01 00:32:09,170,87,5.20,...,2.19,2.75,NaN,0.0,16.75,N,N,,N,N
2,HV0003,B02764,B02764,2020-07-01 00:31:14,2020-07-01 00:32:09,2020-07-01 00:33:39,2020-07-01 01:12:35,87,87,13.43,...,4.88,2.75,NaN,0.0,38.76,N,N,,N,N


## 2. pyspark로 읽기

In [13]:
from pyspark import SparkConf, SparkContext

In [14]:
# Spark 설정
conf = SparkConf().setMaster("local").setAppName("uber-date-trips")
sc = SparkContext(conf=conf)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/18 02:36:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [15]:
%%time
# 데이터 파싱
lines = sc.textFile(filename)
header = lines.first() 
filtered_lines = lines.filter(lambda row:row != header) 
dates = filtered_lines.map(lambda x: x.split(",")[2].split(" ")[0])

CPU times: user 4.73 ms, sys: 2.82 ms, total: 7.55 ms
Wall time: 4.55 s
